In [ ]:
# Technical Indicator Development - Market Research System v1.0
# File: notebooks/development/indicator_development.ipynb
# Created: January 2022
# Purpose: Development and testing of custom technical indicators for Indian stock market

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting parameters
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class TechnicalIndicators:
    """
    Custom Technical Indicators for Indian Stock Market Analysis
    Developed for Market Research System v1.0 (2022)
    """
    
    def __init__(self, data):
        """
        Initialize with stock data DataFrame
        Expected columns: Open, High, Low, Close, Volume
        """
        self.data = data.copy()
        self.close = data['Close']
        self.high = data['High']
        self.low = data['Low']
        self.volume = data['Volume']
        
    def sma(self, period=20):
        """Simple Moving Average"""
        return self.close.rolling(window=period).mean()
    
    def ema(self, period=20):
        """Exponential Moving Average"""
        return self.close.ewm(span=period).mean()
    
    def rsi(self, period=14):
        """Relative Strength Index"""
        delta = self.close.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi
    
    def macd(self, fast=12, slow=26, signal=9):
        """MACD Indicator"""
        ema_fast = self.ema(fast)
        ema_slow = self.ema(slow)
        macd_line = ema_fast - ema_slow
        signal_line = macd_line.ewm(span=signal).mean()
        histogram = macd_line - signal_line
        
        return pd.DataFrame({
            'MACD': macd_line,
            'Signal': signal_line,
            'Histogram': histogram
        })
    
    def bollinger_bands(self, period=20, std_dev=2):
        """Bollinger Bands"""
        sma = self.sma(period)
        std = self.close.rolling(window=period).std()
        
        upper_band = sma + (std * std_dev)
        lower_band = sma - (std * std_dev)
        
        return pd.DataFrame({
            'Upper': upper_band,
            'Middle': sma,
            'Lower': lower_band
        })
    
    def stochastic(self, k_period=14, d_period=3):
        """Stochastic Oscillator"""
        lowest_low = self.low.rolling(window=k_period).min()
        highest_high = self.high.rolling(window=k_period).max()
        
        k_percent = 100 * ((self.close - lowest_low) / (highest_high - lowest_low))
        d_percent = k_percent.rolling(window=d_period).mean()
        
        return pd.DataFrame({
            '%K': k_percent,
            '%D': d_percent
        })
    
    def williams_r(self, period=14):
        """Williams %R"""
        highest_high = self.high.rolling(window=period).max()
        lowest_low = self.low.rolling(window=period).min()
        
        wr = -100 * ((highest_high - self.close) / (highest_high - lowest_low))
        return wr
    
    def atr(self, period=14):
        """Average True Range"""
        high_low = self.high - self.low
        high_close = np.abs(self.high - self.close.shift())
        low_close = np.abs(self.low - self.close.shift())
        
        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = ranges.max(axis=1)
        
        return true_range.rolling(window=period).mean()
    
    def obv(self):
        """On-Balance Volume"""
        obv = np.where(self.close > self.close.shift(1), self.volume,
                      np.where(self.close < self.close.shift(1), -self.volume, 0))
        return pd.Series(obv, index=self.close.index).cumsum()


def fetch_indian_stock_data(symbol, period="1y"):
    """
    Fetch Indian stock data from Yahoo Finance
    Symbol format: RELIANCE.NS, TCS.NS, INFY.NS etc.
    """
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history(period=period)
        return data
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None


def test_indicators_on_nifty50():
    """
    Test indicators on major Indian stocks
    """
    # Top 10 Nifty 50 stocks for testing
    test_stocks = [
        'RELIANCE.NS',  # Reliance Industries
        'TCS.NS',       # Tata Consultancy Services
        'INFY.NS',      # Infosys
        'HDFCBANK.NS',  # HDFC Bank
        'ICICIBANK.NS', # ICICI Bank
        'HINDUNILVR.NS', # Hindustan Unilever
        'ITC.NS',       # ITC
        'KOTAKBANK.NS', # Kotak Mahindra Bank
        'LT.NS',        # Larsen & Toubro
        'BAJFINANCE.NS' # Bajaj Finance
    ]
    
    results = {}
    
    for stock in test_stocks:
        print(f"Testing indicators for {stock}...")
        data = fetch_indian_stock_data(stock, "2y")
        
        if data is not None and len(data) > 50:
            indicators = TechnicalIndicators(data)
            
            # Calculate all indicators
            stock_results = {
                'data': data,
                'SMA_20': indicators.sma(20),
                'SMA_50': indicators.sma(50),
                'EMA_20': indicators.ema(20),
                'RSI': indicators.rsi(14),
                'MACD': indicators.macd(),
                'Bollinger': indicators.bollinger_bands(),
                'Stochastic': indicators.stochastic(),
                'Williams_R': indicators.williams_r(),
                'ATR': indicators.atr(),
                'OBV': indicators.obv()
            }
            
            results[stock] = stock_results
            print(f"✓ Completed {stock}")
        else:
            print(f"✗ Failed to get data for {stock}")
    
    return results


def plot_technical_analysis(stock_symbol, stock_data):
    """
    Create comprehensive technical analysis plots
    """
    fig, axes = plt.subplots(4, 1, figsize=(15, 20))
    fig.suptitle(f'Technical Analysis - {stock_symbol}', fontsize=16, fontweight='bold')
    
    data = stock_data['data']
    indicators = TechnicalIndicators(data)
    
    # Price chart with moving averages and Bollinger Bands
    axes[0].plot(data.index, data['Close'], label='Close Price', linewidth=2)
    axes[0].plot(data.index, stock_data['SMA_20'], label='SMA 20', alpha=0.7)
    axes[0].plot(data.index, stock_data['SMA_50'], label='SMA 50', alpha=0.7)
    
    bb = stock_data['Bollinger']
    axes[0].fill_between(data.index, bb['Upper'], bb['Lower'], alpha=0.2, label='Bollinger Bands')
    axes[0].set_title('Price Action with Moving Averages')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # RSI
    axes[1].plot(data.index, stock_data['RSI'], label='RSI', color='purple')
    axes[1].axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought')
    axes[1].axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold')
    axes[1].set_title('Relative Strength Index (RSI)')
    axes[1].set_ylim(0, 100)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # MACD
    macd = stock_data['MACD']
    axes[2].plot(data.index, macd['MACD'], label='MACD', color='blue')
    axes[2].plot(data.index, macd['Signal'], label='Signal', color='red')
    axes[2].bar(data.index, macd['Histogram'], label='Histogram', alpha=0.7, color='green')
    axes[2].set_title('MACD Indicator')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    # Volume and OBV
    axes[3].bar(data.index, data['Volume'], alpha=0.3, label='Volume')
    ax_obv = axes[3].twinx()
    ax_obv.plot(data.index, stock_data['OBV'], color='orange', label='OBV')
    axes[3].set_title('Volume and On-Balance Volume (OBV)')
    axes[3].legend(loc='upper left')
    ax_obv.legend(loc='upper right')
    axes[3].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()


def generate_signals(stock_data):
    """
    Generate basic buy/sell/hold signals based on technical indicators
    """
    data = stock_data['data']
    latest_close = data['Close'].iloc[-1]
    latest_rsi = stock_data['RSI'].iloc[-1]
    latest_macd = stock_data['MACD']['MACD'].iloc[-1]
    latest_signal = stock_data['MACD']['Signal'].iloc[-1]
    
    signals = []
    
    # RSI signals
    if latest_rsi < 30:
        signals.append(("RSI", "BUY", f"RSI at {latest_rsi:.2f} - Oversold"))
    elif latest_rsi > 70:
        signals.append(("RSI", "SELL", f"RSI at {latest_rsi:.2f} - Overbought"))
    else:
        signals.append(("RSI", "HOLD", f"RSI at {latest_rsi:.2f} - Neutral"))
    
    # MACD signals
    if latest_macd > latest_signal:
        signals.append(("MACD", "BUY", f"MACD above signal line"))
    else:
        signals.append(("MACD", "SELL", f"MACD below signal line"))
    
    # Moving average signals
    sma_20 = stock_data['SMA_20'].iloc[-1]
    sma_50 = stock_data['SMA_50'].iloc[-1]
    
    if latest_close > sma_20 > sma_50:
        signals.append(("MA", "BUY", f"Price above both MAs - Uptrend"))
    elif latest_close < sma_20 < sma_50:
        signals.append(("MA", "SELL", f"Price below both MAs - Downtrend"))
    else:
        signals.append(("MA", "HOLD", f"Mixed signals from MAs"))
    
    return signals


# Main execution for testing
if __name__ == "__main__":
    print("=== Technical Indicator Development - Market Research System v1.0 ===")
    print("Testing custom technical indicators on Indian stocks...")
    print("=" * 70)
    
    # Test indicators on selected stocks
    results = test_indicators_on_nifty50()
    
    # Display results for first stock
    if results:
        first_stock = list(results.keys())[0]
        print(f"\nDetailed analysis for {first_stock}:")
        print("=" * 50)
        
        # Plot technical analysis
        plot_technical_analysis(first_stock, results[first_stock])
        
        # Generate signals
        signals = generate_signals(results[first_stock])
        print("\nGenerated Signals:")
        for indicator, action, reason in signals:
            print(f"{indicator}: {action} - {reason}")
        
        print("\n=== Indicator Development Complete ===")
        print("All custom indicators tested successfully!")
        print("Ready for integration into main system...")